In [7]:
#test scrape
import json
import re
import requests
from bs4 import *
import time


In [8]:
text = requests.get("https://www.makro.co.za/appliances/fridges-freezers/commercial-fridges/c/AAA")
soup = BeautifulSoup(text, 'lxml')


FeatureNotFound: Couldn't find a tree builder with the features you requested: lxml. Do you need to install a parser library?